<a href="https://colab.research.google.com/github/kpacker77/CSE158_Assign2/blob/main/Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data_path="/content/drive/MyDrive/CSE158_Assignment2"

In [4]:
# change directory to the project folder
%cd $data_path
# sanity check this works
import os
print("Now working in:", os.getcwd())
!ls  # list files in the folder

/content/drive/MyDrive/CSE158_Assignment2
Now working in: /content/drive/MyDrive/CSE158_Assignment2
modcloth_final_data.json.gz  renttherunway_final_data.json.gz
